#librerias y ubicacion directorio

In [1]:
from google.colab import drive #Acceso Drive
import os #Acceder a directorios
import pandas as pd #manejo de dataframes
import folium #mostrar mapa con geocolocacion 
import numpy as np #Numerical Python
import librosa
import IPython.display as ipd

In [2]:
#Damos acceso a colab a nuestro Drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
#Nos ubicamos en la direccion de interes

ruta_audios='/content/drive/MyDrive/Jaguas_2018' #direccion raiz de nuestro proyecto actual
ruta_py="/content/drive/MyDrive/Semillero_IA/acoustic_exploration"

In [4]:
"""Cuenta la cantidad de archivos en cada carpeta de un directorio"""
total_archivos = 0
for raiz, carpetas, archivos in os.walk(ruta_audios):
  carpetas.sort()
  # Obtener la cantidad de archivos en la carpeta actual
  cantidad_archivos = len(archivos)  
  # Sumar la cantidad de archivos actual a la variable total_archivos
  total_archivos += cantidad_archivos      
  # Imprimir el resultado
  print(f"Grabadora: {os.path.basename(raiz)} Cantidad Grabaciones: {cantidad_archivos}")
print(f"Total grabaciones de todas las grabadoras: {total_archivos}")

Total grabaciones de todas las grabadoras: 0


#Creacion Data Frame df_audios



In [ ]:
# Crear un dataframe vacío para guardar la información de los archivos
df_audios = pd.DataFrame(columns=['Recording','Recorder', 'Date', 'Time', 'Path'])

# Recorrer las carpetas y archivos dentro de la ruta principal
for raiz, carpetas, archivos in os.walk(ruta_audios):
    carpetas.sort()
    # Recorrer los archivos de cada carpeta
    for archivo in archivos:
        # Obtener la ruta completa del archivo
        ruta_archivo = os.path.join(raiz, archivo)
        ruta_archivo = ruta_archivo.replace(ruta_audios, '')
        # Obtener el nombre del archivo sin la extensión
        nombre_archivo, extension = os.path.splitext(archivo)
        
        
        # Separar el nombre del archivo en partes utilizando el caracter "_"
        partes_nombre = nombre_archivo.split("_")
        
        # Obtener los valores deseados de cada parte del nombre del archivo
        recorder = partes_nombre[0]
        fecha = partes_nombre[1]
        hora = partes_nombre[2]
        nombre_archivo = archivo
        # Crear un diccionario con los valores del archivo actual
        datos_archivo = {
            'Recording': nombre_archivo,
            'Recorder': recorder,
            'Date': fecha,
            'Time': hora,
            'Path': ruta_archivo
        }
        
        # Agregar el diccionario al dataframe
        df_audios = df_audios.append(datos_archivo, ignore_index=True)


# Convertir la columna 'Date' al formato 'yyyy/mm/dd'
df_audios['Date'] = df_audios['Date'].apply(lambda x: pd.to_datetime(x, format='%Y%m%d').strftime('%Y/%m/%d'))

# Convertir la columna 'Time' al formato 'hh:mm:ss'
df_audios['Time'] = df_audios['Time'].apply(lambda x: pd.to_datetime(x, format='%H%M%S').strftime('%H:%M:%S'))

#Cargamos informacion adicional de las grabadoras
df_grabadoras = pd.read_csv(ruta_py+'/df_grabadoras.csv')
df_lluvia = pd.read_csv(ruta_py+'/Base_Etiquetas_LLuvia_26Abr.csv')
df_lluvia = df_lluvia.rename(columns={'Filename': 'Recording'})
df_lluvia = df_lluvia.drop(columns=['Site'])

#hacemos un Marge con el dataframe anteriormente creado, a base de recorrer los archivos
df_audios = pd.merge(df_audios, df_grabadoras, on="Recorder")
df_audios = pd.merge(df_audios, df_lluvia, on="Recording")

Se han truncado las últimas 5000 líneas del flujo de salida.
<ipython-input-6-5fcc0f067436>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_audios = df_audios.append(datos_archivo, ignore_index=True)
<ipython-input-6-5fcc0f067436>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_audios = df_audios.append(datos_archivo, ignore_index=True)
<ipython-input-6-5fcc0f067436>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_audios = df_audios.append(datos_archivo, ignore_index=True)
<ipython-input-6-5fcc0f067436>:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_audios = df_audios.append(datos_archivo, ignore_index=True)
<ipytho

In [ ]:
df_audios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20068 entries, 0 to 20067
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Recording           20068 non-null  object 
 1   Recorder            20068 non-null  object 
 2   Date                20068 non-null  object 
 3   Time                20068 non-null  object 
 4   Path                20068 non-null  object 
 5   Municipio           20068 non-null  object 
 6   Vereda              20068 non-null  object 
 7   Latitud             20068 non-null  float64
 8   Longitud            20068 non-null  float64
 9   Elevacion           20068 non-null  int64  
 10  Habitat             20068 non-null  object 
 11  Total_grabaciones   20068 non-null  int64  
 12  Rain_occurrence     20068 non-null  int64  
 13  Rain_Intesity       20068 non-null  int64  
 14  Intensity_Category  20068 non-null  object 
dtypes: float64(2), int64(4), object(9)
memory usage: 2.4+

In [ ]:
df_audios.head()

,Recording,Recorder,Date,Time,Path,Municipio,Vereda,Latitud,Longitud,Elevacion,Habitat,Total_grabaciones,Rain_occurrence,Rain_Intesity,Intensity_Category
0,G03_20180511_120000.wav,G03,2018/05/11,12:00:00,/G03_m/G03_20180511_120000.wav,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest,565,0,0,No_rain
1,G03_20180511_121600.wav,G03,2018/05/11,12:16:00,/G03_m/G03_20180511_121600.wav,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest,565,0,0,No_rain
2,G03_20180511_123200.wav,G03,2018/05/11,12:32:00,/G03_m/G03_20180511_123200.wav,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest,565,0,0,No_rain
3,G03_20180511_124800.wav,G03,2018/05/11,12:48:00,/G03_m/G03_20180511_124800.wav,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest,565,0,0,No_rain
4,G03_20180511_130400.wav,G03,2018/05/11,13:04:00,/G03_m/G03_20180511_130400.wav,Alejandria,El Respaldo,6.4137,-75.0366,1322,Forest,565,0,0,No_rain


#Busqueda de Audios

In [ ]:
print(df_audios['Intensity_Category'].value_counts())
print(df_audios['Rain_occurrence'].value_counts())
print(df_audios['Rain_Intesity'].value_counts())


No_rain     16964
Light        2062
Moderate      754
Heavy         288
Name: Intensity_Category, dtype: int64
0    16964
1     3104
Name: Rain_occurrence, dtype: int64
0    16964
1     2062
2      754
3      288
Name: Rain_Intesity, dtype: int64


In [ ]:
for i, (index, row) in enumerate(df_audios.loc[(df_audios["Rain_Intesity"] == 1)].iterrows()):
    if i == 2: # salir después de 2 iteraciones
        break
    audio_path = ruta_audios + row["Path"]
    print(f"Iteración {i+1}: Abriendo archivo de audio en {audio_path}")
    print(row)
    audio, sr = librosa.load(audio_path)
    ipd.display(ipd.Audio(audio, rate=sr))


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Iterar sobre todos los elementos que cumplen las condiciones de "Rain_Intesity" igual a 1 o 2 y "Habitat" igual a "Forest"
for i, (index, row) in enumerate(df_audios.loc[(df_audios["Rain_Intesity"].isin([1, 2])) & (df_audios["Habitat"] == "Forest")].iterrows()):
    if i == 2: # salir después de 2 iteraciones
      break
    audio_path = ruta_audios + row["Path"]
    print(f"Iteración {i+1}: Abriendo archivo de audio en {audio_path}")
    print(row)
    audio, sr = librosa.load(audio_path)
    ipd.display(ipd.Audio(audio, rate=sr))



Output hidden; open in https://colab.research.google.com to view.